In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [2]:
submissson = pd.read_csv('submission.csv')

In [3]:
from preprocess import preprocess

In [4]:
test, colnames = preprocess("test.csv")

In [5]:
test.head()

,ego_id,u,v,t,x1,x2,x3,city_dist,university_dist,sex,...,gen_1,sex2_0,sex2_1,sex2_2,city_0,city_1,city_2,university_0,university_1,university_2
0,8,20,19,185.7,3.839089e-04,0.0,0.0,0,284912378,0,...,True,False,False,True,False,False,True,True,False,False
1,8,10,19,158.3,1.046045e+00,0.0,0.0,0,-1,0,...,False,False,False,True,False,False,True,True,False,False
2,8,2,19,594.5,7.529229e-01,0.0,0.0,0,188425734,0,...,True,False,False,True,False,False,True,True,False,False
3,8,8,19,NaN,1.751368e-06,0.0,0.0,-1,-1,1,...,False,True,False,False,True,False,False,True,False,False
4,8,20,2,164.1,3.034230e-13,0.0,0.0,0,473338112,0,...,True,False,False,True,False,False,True,True,False,False


In [6]:
colnames

Index(['x2', 'x3', 'city_dist', 'university_dist', 'sex', 'school_dist',
       'age_dist', 'age_mean', 'school', 'sum', 'u_v', 'gen_0', 'gen_1',
       'sex2_0', 'sex2_1', 'sex2_2', 'city_0', 'city_1', 'city_2',
       'university_0', 'university_1', 'university_2'],
      dtype='object')

In [7]:
import pickle

In [8]:
# Загрузка модели
with open ('random_forest.pkl', 'rb') as f:
    myModel = pickle.load(f)

In [9]:
with open('catboost_full_model.pkl', 'rb') as f:
    cb = pickle.load(f)

In [10]:
with open('line_reg_model.pkl', 'rb') as f:
    lin_reg = pickle.load(f)

C:\Users\Lev\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
from config import *
from modelerWG import predict, rmse

fores_pred = predict(test, myModel, colnames=colnames)
cb_pred = predict(test, cb, colnames=colnames)

In [12]:
learn = pd.DataFrame({'forest':fores_pred, 'cb':cb_pred})

In [13]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3, include_bias=False)
poly_features = poly.fit_transform(learn)

In [23]:
poly_features.reshape(9, -1).shape

(9, 34678164)

In [26]:
preds_randmforest = predict(pd.DataFrame(poly_features), lin_reg, colnames=pd.DataFrame(poly_features).columns)

In [27]:
test["preds"] = preds_randmforest

In [33]:
# Соединение Submission и Test
res = submissson.merge(test, on=["ego_id", "u", "v"], how="inner")

In [34]:
# Вывод только нужного
res = res[['ego_id',"u","v", "preds", "x1_x"]]

In [ ]:
res.to_csv('submission2.csv', index = False)

In [36]:
res

,ego_id,u,v,preds,x1_x
0,8,0,93,0.019135,0.000000
1,8,0,143,0.156375,0.000000
2,8,0,151,1.551773,1.606742
3,8,1,24,0.300220,0.026496
4,8,5,4,0.505534,0.159857
...,...,...,...,...,...
693927,1709396984676,73,23,1.281413,1.562170
693928,1709396984676,74,68,0.566864,0.454210
693929,1709396984676,77,28,1.213811,0.078504
693930,1709396984676,79,38,0.956417,0.981812


In [37]:
from sklearn.metrics import mean_squared_error
mean_squared_error(res['x1_x'], res['preds'], squared=False)

1.2972360479346356